In [ ]:
!pip install -q google-adk

In [ ]:
import os
try:
    from google.colab import userdata
    os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")
except ImportError:
    pass  # Not running in Colab; uses env var already set

# Travel Assistant Agent

Build an agent that helps users plan trips. Your agent will have these tools:

1. **search_flights** — Find flights between two cities on a given date
2. **check_visa_requirements** — Look up visa requirements for a destination country
3. **convert_currency** — Convert an amount between two currencies

Tools return hardcoded/fake data — the goal is designing the **interface** between the model and your code.

## Setup

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.genai import types

## TODO 1: Define your tools

Just write plain Python functions. The ADK infers the schema from:
- **Type hints** → parameter types
- **Docstring** → tool description

The first tool is done for you as an example. Implement the remaining two.

In [ ]:
# Example tool (complete)

def search_flights(origin: str, destination: str, date: str) -> dict:
    """Search for available flights between two cities on a given date.

    Args:
        origin: Departure city (e.g. "New York").
        destination: Arrival city (e.g. "London").
        date: Travel date in YYYY-MM-DD format.

    Returns:
        dict: status and flight options or error message.
    """
    return {
        "status": "success",
        "report": (
            f"Found 3 flights from {origin} to {destination} on {date}: "
            f"1) Departure 08:00, Arrival 20:00, $450 economy. "
            f"2) Departure 14:30, Arrival 02:30+1, $380 economy. "
            f"3) Departure 22:00, Arrival 10:00+1, $820 business."
        ),
    }


def check_visa_requirements(passport_country: str, destination_country: str) -> dict:
    """Check visa requirements for traveling to a destination country.

    Args:
        passport_country: The traveler's passport country (e.g. "United States").
        destination_country: The country they want to visit (e.g. "Japan").

    Returns:
        dict: status and visa requirement details or error message.
    """
    # TODO: implement — return a dict with "status" and "report" keys
    # Hint: return hardcoded info like visa-free duration, visa type needed, etc.
    pass


def convert_currency(amount: float, from_currency: str, to_currency: str) -> dict:
    """Convert an amount from one currency to another.

    Args:
        amount: The amount of money to convert.
        from_currency: Source currency code (e.g. "USD").
        to_currency: Target currency code (e.g. "EUR").

    Returns:
        dict: status and conversion result or error message.
    """
    # TODO: implement — return a dict with "status" and "report" keys
    # Hint: use a hardcoded exchange rate and calculate the converted amount
    pass

## TODO 2: Define your agent

In [ ]:
agent = Agent(
    model="gemini-3-flash-preview",
    name="travel_assistant",
    # TODO: Write an instruction that gives the agent a travel-advisor persona.
    # Hint: describe what the agent can help with, its tone, and any guidelines.
    instruction="""You are a helpful travel assistant.

Replace this with your agent's persona and guidelines.
""",
    tools=[search_flights, check_visa_requirements, convert_currency],
)

## Run your agent

Once you've filled in the TODOs above, run this cell to test your agent!

In [ ]:
runner = InMemoryRunner(agent=agent, app_name="travel_assistant")
runner.auto_create_session = True
user_content = types.Content(role="user", parts=[types.Part.from_text(text="I want to travel from San Francisco to Tokyo next month. What flights are available and do I need a visa?")])

final_text = ""
async for event in runner.run_async(user_id="user1", session_id="session1", new_message=user_content):
    if event.content and event.content.parts:
        for part in event.content.parts:
            if part.function_call:
                print(f"🔧 Tool call: {part.function_call.name}({dict(part.function_call.args)})")
            if part.function_response:
                print(f"  ↳ Response: {part.function_response.name} returned")
            if part.text:
                final_text = part.text

print(f"\n{'='*60}\nFinal response:\n{'='*60}\n{final_text}")

## Tips

- **Docstrings matter** — the ADK uses them as tool descriptions. Be specific about what each parameter means.
- **Fake data is fine** — the goal is designing the agent's interface, not building a real booking engine.
- **Test edge cases** — what happens when a user asks about a city your tools don't know about?

### Stretch Goals
- Add a `get_weather_forecast` tool that returns weather for a destination
- Add a `suggest_itinerary` tool that recommends activities for a city
- Handle multi-leg trips (e.g. "fly from SF to Tokyo, then to Seoul")